In [1]:
import utils
import os
import datetime
from tqdm.notebook import trange
from IPython.display import display
import pandas as pd
from dec_tswap.path_table import PathTable
import manavlib.io.xml_io as xml_io

# Experiments

## Configuration

In [2]:
TASK_SUFFIX = "_task.xml"  # Set task file naming convention with a suffix (_task.xml).
MAP_FILE = "map.xml"  # Specify the name for the map file.
RESULT_FILE = "result.txt"  # Define a name for the result file.

TASK_DIR = (
    "../tasks/example/"  # Define the directory path where task files are located.
)
CONFIG_FILE = "dec_tswap_config.xml"  # Select the algorithm (by choosing the config file) to be used (dec_tswap in this case).
RESULT_DIR = "./results/"  # Define the directory path for the result file.


AGENTS_FROM = 10  # Starting number of agents for the experiment.
AGENTS_TO = 60  # Maximum number of agents for the experiment.
AGENTS_STEP = 10  # Increment step for the number of agents in experiment run.
TASK_NUM = 10  # Number of tasks to run.

In [3]:
map_path = os.path.join(TASK_DIR, MAP_FILE)
config_path = os.path.join(TASK_DIR, CONFIG_FILE)

# Read experiment and algorithm parameters from the XML configuration file.
exp_params, alg_params = xml_io.read_xml_config(config_path)

# Load map data from the XML map file.
h, w, cs, grid_map = xml_io.read_xml_map(map_path)

## Experiemnts Execution

In [4]:
result_path = os.path.join(RESULT_DIR, f"{RESULT_FILE}")
result_file = open(result_path, "w")
result_file.write(utils.Summary.header() + "\n")
for task_id in (
    pbar_task := trange(
        TASK_NUM, bar_format="{desc:<20}{percentage:3.0f}%|{bar}{r_bar}"
    )
):
    pbar_task.set_description(f"{task_id+1} / {TASK_NUM} task")
    pbar_task.refresh()
    task_file = f"{task_id}{TASK_SUFFIX}"
    task_path = os.path.join(TASK_DIR, task_file)
    default_params, starts, goals, ag_params = xml_io.read_xml_agents(task_path)
    path_table = PathTable(grid_map, goals[:AGENTS_TO])
    for agents_num in (
        pbar_agent := trange(
            AGENTS_FROM,
            AGENTS_TO + 1,
            AGENTS_STEP,
            colour="#FFA500",
            bar_format="{desc:<20}{percentage:3.0f}%|{bar}{r_bar}",
        )
    ):
        pbar_agent.set_description(
            f"Task {task_id}. {agents_num}/{AGENTS_TO} agents",
        )
        pbar_agent.refresh()

        summary, steps_log, neighbors_log = utils.run_experiment(
            starts,
            goals,
            grid_map,
            cs,
            agents_num,
            utils.get_agent_type(type(alg_params)),
            ag_params,
            alg_params,
            exp_params,
            path_table,
            False,
        )

        result_file.write(str(summary) + "\n")
result_file.close()

                      0%|          | 0/10 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

...

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

                      0%|          | 0/6 [00:00<?, ?it/s]

## Preliminarily Results Processing

In [5]:
results = pd.read_table(result_path, skipinitialspace=True, sep=" ")
results.head()

,success,collision,collision_obst,makespan,flowtime,runtime,mean_groups,mean_groups_size,number
0,1,0,0,76,259,0.064,9.329,1.08,10
1,1,0,0,82,502,0.194,15.537,1.29,20
2,1,0,0,87,519,0.487,22.000,1.38,30
3,1,0,0,46,444,0.475,26.500,1.52,40
4,1,0,0,73,771,1.200,30.644,1.64,50


In [6]:
gr_results = results.groupby("number")
sr = pd.DataFrame()
sr["success rate"] = gr_results["success"].mean()

success_results = results.drop(results[results.success < 1].index)

sr["flowtime"] = gr_results["flowtime"].mean()
sr["makespan"] = gr_results["makespan"].mean()
sr["runtime"] = gr_results["runtime"].mean()
sr_style = sr.style.format(
    {
        "success rate": "{:.0%}",
        "flowtime": "{:.1f}",
        "makespan": "{:.1f}",
        "runtime": "{:.3f}",
    }
)
sr_style

,success rate,flowtime,makespan,runtime
number,,,,
10,100%,226.6,59.9,0.055
20,100%,387.2,66.1,0.150
30,100%,523.7,72.6,0.356
40,100%,581.3,73.1,0.724
50,100%,765.7,71.5,1.282
60,100%,831.9,67.0,1.920
